# Extractice QA :  SQuAD

This is a simple notebook to download and store the SQuAD v1.1 extractive qa dataset. We will take it from Huggingface datasets libray (<3) and turn it into one of the two formats that classy is able to parse (i.e. jsonl).

After creating this dataset you can train the model by exectuing the following bash command
```bash
classy train qa data/qa/squad -n my_firt_squad_run
```

In [1]:
! pip install datasets

In [2]:
import json
import os

from datasets import load_dataset
from tqdm.notebook import tqdm

In [3]:
# here we load the dataset dataset from "datasets"
dataset = load_dataset("squad")

Reusing dataset squad (/home/edobobo/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [5]:
dataset["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [6]:
# let's create a repositiory that will contains the dataset splits
import os
dir_path = "squad-v1.1"
os.mkdir(dir_path)

FileExistsError: [Errno 17] File exists: 'squad-v1.1'

In [7]:
# if you want the output format to be tab separated decomment the second line
output_format = "jsonl"
# output_format = "tsv"

In [11]:
from collections import Counter


for k in ['train', 'validation']:

    with open(f'{dir_path}/{k}.{output_format}', 'w') as f:

        for sample in tqdm(dataset[k], desc=f"Processing {k}"):

            squad_id = sample["id"]
            title = sample["title"]
            context = sample["context"]
            question = sample["question"].lstrip()  # suggested by huggingface to remove left-side white spaces
            
            assert len(sample["answers"]["answer_start"]) == len(sample["answers"]["text"])
            
            # save original answer, and encode as start and end only first
            full_answers = sample["answers"]
            answers_text = sample["answers"]["text"][0]
            answers_start = sample["answers"]["answer_start"][0]
            answers_end = answers_start + len(answers_text)
            assert answers_text == context[answers_start: answers_end]
            
            # write
            f.write(json.dumps(
                dict(
                        context=context,
                        question=question,
                        answer_start=answers_start,
                        answer_end=answers_end,
                        squad_id=squad_id,
                        title=title,
                        full_answers=full_answers,
                    )
            ) + "\n")

Processing train:   0%|          | 0/87599 [00:00<?, ?it/s]

Processing validation:   0%|          | 0/10570 [00:00<?, ?it/s]

In [12]:
! head -5 $dir_path/train.$output_format

{"context": "Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.", "question": "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?", "answer_start": 515, "answer_end": 541, "squad_id": "5733be284776f41900661182", "title": "University_of_Notre_Dame", "full_answers": {"text": ["Saint Bernadette Soubirous"], "answer_start": [515]}}
{"

In [13]:
! head -5 $dir_path/validation.$output_format

{"context": "Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24\u201310 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the \"golden anniversary\" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as \"Super Bowl L\"), so that the logo could prominently feature the Arabic numerals 50.", "question": "Which NFL team represented the AFC at Super Bowl 50?", "answer_start": 177, "answer_end": 191, "squad_id": "56be4db0acb8001400a502ec", "title": "Super_Bowl_50", "full_answers": {"text": [